# ETL/Data Warehousing Project—Global GDP and Sales Analysis

### Extracting the GDP data from WIkipedia - Bronze Layer

In [1]:
# Import required libraries for data handling and web scraping
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [2]:
# Retrieving and parsing the Wikipedia page containing GDP data
headers = {
    "User-Agent": "MyDWProject/1.0 (DWskills)"
}
url = 'https://en.wikipedia.org/wiki/List_of_countries_by_GDP_(nominal)'
response = requests.get(url, headers=headers)
response.raise_for_status()
html = response.text
soup = BeautifulSoup(html, "html.parser")

In [3]:
# Extracts rows and cell data from the Wikipedia GDP table
table = soup.find("table", class_="wikitable")

rows = []
for tr in table.find_all("tr"):
    cells = [td.get_text(strip=True) for td in tr.find_all(["th", "td"])]
    if cells:
        rows.append(cells)


In [4]:
# Automatically extract all tables from the HTML using pandas
tables = pd.read_html(html)
tables

[                                                   0
 0  Largest economies in the world by GDP (nominal...,
                                                    0  \
 0  .mw-parser-output .legend{page-break-inside:av...   
 
                                                    1  
 0  $250–500 billion $100–250 billion $50–100 bill...  ,
     Country/Territory IMF (2025)[6] World Bank (2024)[7]  \
 0               World     117165394            111326370   
 1       United States      30615743             28750956   
 2          China[n 1]      19398577             18743803   
 3             Germany       5013574              4685593   
 4               Japan       4279828              4027598   
 ..                ...           ...                  ...   
 217          Kiribati           321                  308   
 218  Marshall Islands           302                  290   
 219             Nauru           172                  163   
 220        Montserrat          —N/a                

In [5]:
# checking data type
type(tables) 


list

In [6]:
# checking how many tables were found
len(tables)


7

In [7]:
#To select and rename the table needed for the task(the third table)
CountriesGDP_df = tables[2]

In [8]:
# Get and print the page's last modification date
timestamp = soup.find("li", id="footer-info-lastmod").text
print(timestamp)


 This page was last edited on 5 February 2026, at 13:01 (UTC).


In [9]:
CountriesGDP_df['source_url'] = url

The source URL is added temporarily for data lineage and validation, then removed during transformation to keep the final dataset clean and normalized


### SILVER LAYER

In [10]:
# Check the structure of the GDP DataFrame to inspect data types and missing values
print(CountriesGDP_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 222 entries, 0 to 221
Data columns (total 5 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   Country/Territory         222 non-null    object
 1   IMF (2025)[6]             222 non-null    object
 2   World Bank (2024)[7]      222 non-null    object
 3   United Nations (2024)[8]  222 non-null    object
 4   source_url                222 non-null    object
dtypes: object(5)
memory usage: 8.8+ KB
None


In [11]:
# Total missing per column
print(CountriesGDP_df.isnull().sum())

# Percentage missing per column
missing_pct = CountriesGDP_df.isnull().mean() * 100
print(missing_pct)


Country/Territory           0
IMF (2025)[6]               0
World Bank (2024)[7]        0
United Nations (2024)[8]    0
source_url                  0
dtype: int64
Country/Territory           0.0
IMF (2025)[6]               0.0
World Bank (2024)[7]        0.0
United Nations (2024)[8]    0.0
source_url                  0.0
dtype: float64


Checking for missing values returned 0 despite visible missing data points. This is because the missing values were stored as string literals(—N/a) which python couldn't capture as Standard Null values.

In [12]:
# Duplicate rows
duplicates = CountriesGDP_df[CountriesGDP_df.duplicated()]
print(duplicates)

# Check duplicates in a key column, e.g., 'Country'
if 'Country' in CountriesGDP_df.columns:
    print(CountriesGDP_df['Country/Territory'][CountriesGDP_df['Country'].duplicated()])


Empty DataFrame
Columns: [Country/Territory, IMF (2025)[6], World Bank (2024)[7], United Nations (2024)[8], source_url]
Index: []


In [13]:
# Display the total number of unique values in the country column
if 'Country' in CountriesGDP_df.columns:
    print("Number of unique countries:", CountriesGDP_df['Country'].nunique())
    print("All unique countries:\n", CountriesGDP_df['Country'].unique())


In [14]:
# identify unusually low (negative) or extremely high GDP values that may indicate errors
if 'GDP' in CountriesGDP_df.columns:
    print(CountriesGDP_df[CountriesGDP_df['GDP'] < 0])
    print(CountriesGDP_df[CountriesGDP_df['GDP'] > 1e13])  # adjust threshold


In [15]:
# Remove leading and trailing whitespace from all string (object) columns
str_cols = CountriesGDP_df.select_dtypes(include='object').columns
for col in str_cols:
    CountriesGDP_df[col] = CountriesGDP_df[col].str.strip()

# Count any remaining empty strings in these columns
print("Number of empty strings in each column:\n", (CountriesGDP_df[str_cols] == '').sum())



Number of empty strings in each column:
 Country/Territory           0
IMF (2025)[6]               0
World Bank (2024)[7]        0
United Nations (2024)[8]    0
source_url                  0
dtype: int64


In [16]:
# Display all column names in the GDP DataFrame
print("Columns in the table:")
print(CountriesGDP_df.columns.tolist())


Columns in the table:
['Country/Territory', 'IMF (2025)[6]', 'World Bank (2024)[7]', 'United Nations (2024)[8]', 'source_url']


I noticed that the column names for this tables is quite dynamic, it changes from time to time, for this reason, I have to create a loop that would adapt to this change. Using the regular rename column method will not work in this case especially when a rereun or a refresh is done because I can't hard-code the column names.

In [17]:
import re

# Creating a loop to build the mapping dynamically
rename_map = {}

for col in CountriesGDP_df.columns:
    #  Clean the Country column
    if 'Country' in col:
        rename_map[col] = 'Country'
    
    # Extracting the Source and Year for GDP columns
    else:
       
        year_match = re.search(r'(\d{4})', col)
        if year_match:
            year = year_match.group(1)
            
            if 'IMF' in col:
                rename_map[col] = f'GDP_IMF_{year}'
            elif 'World Bank' in col:
                rename_map[col] = f'GDP_WB_{year}'
            elif 'United Nations' in col:
                rename_map[col] = f'GDP_UN_{year}'

# Apply the result
CountriesGDP_RenamedColumns = CountriesGDP_df.rename(columns=rename_map)

In [18]:
CountriesGDP_RenamedColumns

,Country,GDP_IMF_2025,GDP_WB_2024,GDP_UN_2024,source_url
0,World,117165394,111326370,100834796,https://en.wikipedia.org/wiki/List_of_countrie...
1,United States,30615743,28750956,29298000,https://en.wikipedia.org/wiki/List_of_countrie...
2,China[n 1],19398577,18743803,18743802,https://en.wikipedia.org/wiki/List_of_countrie...
3,Germany,5013574,4685593,4659929,https://en.wikipedia.org/wiki/List_of_countrie...
4,Japan,4279828,4027598,4026211,https://en.wikipedia.org/wiki/List_of_countrie...
...,...,...,...,...,...
217,Kiribati,321,308,343,https://en.wikipedia.org/wiki/List_of_countrie...
218,Marshall Islands,302,290,281,https://en.wikipedia.org/wiki/List_of_countrie...
219,Nauru,172,163,187,https://en.wikipedia.org/wiki/List_of_countrie...
220,Montserrat,—N/a,—N/a,81,https://en.wikipedia.org/wiki/List_of_countrie...


In [19]:
# Remove brackets and their contents from "Country", then strip extra spaces. All country names are now properly formatted
CountriesGDP_RenamedColumns['Country'] = CountriesGDP_RenamedColumns['Country'] \
    .str.replace(r'\[.*?\]', '', regex=True) \
    .str.strip()


In [20]:
# Identify GDP-related columns and clean themby removing unwanted characters
GDP_columns = [col for col in CountriesGDP_RenamedColumns.columns if col.startswith('GDP')]

for col in GDP_columns:
    # Ensure the column is treated as a string before cleaning
    CountriesGDP_RenamedColumns[col] = (
        CountriesGDP_RenamedColumns[col]
        .astype(str)
        .str.replace(r'\[.*?\]', '', regex=True)  # removes wiki citations like [6]
        .str.replace(r'\(.*?\)', '', regex=True)  # removes parentheses/notes like (est.)
        .str.replace(',', '')                     # removes commas
        .str.strip()                              # removes extra spaces
    )
    
    # Convert to numeric
    CountriesGDP_RenamedColumns[col] = pd.to_numeric(CountriesGDP_RenamedColumns[col], errors='coerce')

In [21]:
## To replace NA with actual nan so Python can detect and treat as a null value
import numpy as np

# 1. Update the reference to use your new variable name
CountriesGDP = CountriesGDP_RenamedColumns

# 2. Dynamically grab all GDP columns (future-proofs against year changes)
GDP_columns = [col for col in CountriesGDP.columns if col.startswith('GDP')]

for col in GDP_columns:
    # Use a regex-based approach to catch anything starting with '—' or 'N/A'
    # This handles that long CSS string and other weird Wikipedia placeholders
    CountriesGDP[col] = (
        CountriesGDP[col]
        .replace([r'^—.*', r'^N/A$', r'^-$', r'^\s*$'], np.nan, regex=True)
    )

# 3. Final Step: Ensure columns are floats for calculation
CountriesGDP[GDP_columns] = CountriesGDP[GDP_columns].apply(pd.to_numeric, errors='coerce')

In [22]:
# Rechecking for Null values
CountriesGDP[CountriesGDP.isna().any(axis=1)]

,Country,GDP_IMF_2025,GDP_WB_2024,GDP_UN_2024,source_url
22,Taiwan,884387.0,NaN,NaN,https://en.wikipedia.org/wiki/List_of_countrie...
56,Cuba,NaN,107352.0,252063.0,https://en.wikipedia.org/wiki/List_of_countrie...
139,Syria,NaN,23623.0,17254.0,https://en.wikipedia.org/wiki/List_of_countrie...
140,North Korea,NaN,NaN,17199.0,https://en.wikipedia.org/wiki/List_of_countrie...
151,Yemen,NaN,21606.0,14304.0,https://en.wikipedia.org/wiki/List_of_countrie...
153,Channel Islands,NaN,12508.0,NaN,https://en.wikipedia.org/wiki/List_of_countrie...
156,Monaco,NaN,11126.0,11126.0,https://en.wikipedia.org/wiki/List_of_countrie...
159,Bermuda,NaN,9234.0,8840.0,https://en.wikipedia.org/wiki/List_of_countrie...
161,New Caledonia,NaN,8549.0,8549.0,https://en.wikipedia.org/wiki/List_of_countrie...
163,Cayman Islands,NaN,7241.0,7657.0,https://en.wikipedia.org/wiki/List_of_countrie...


Null Values can now be detected after the last step

In [23]:
#Count the number of rows with missing values
CountriesGDP[CountriesGDP.isna().any(axis=1)].shape[0]

28

#### In a real world situation, I would try to resolve the missing values by integrating other economic data sources with a more comprehensive information. For this task, those sources were not utilized due to licensing, usage restriction and API access consideration. To avoid future compliance issues, those sources were not accessed and missing GDP values were stored as NULL.


In [24]:
#Delete url column
CountriesGDP = CountriesGDP_RenamedColumns.drop(columns=['source_url'])


In [25]:
display(CountriesGDP)

,Country,GDP_IMF_2025,GDP_WB_2024,GDP_UN_2024
0,World,117165394.0,111326370.0,100834796.0
1,United States,30615743.0,28750956.0,29298000.0
2,China,19398577.0,18743803.0,18743802.0
3,Germany,5013574.0,4685593.0,4659929.0
4,Japan,4279828.0,4027598.0,4026211.0
...,...,...,...,...
217,Kiribati,321.0,308.0,343.0
218,Marshall Islands,302.0,290.0,281.0
219,Nauru,172.0,163.0,187.0
220,Montserrat,NaN,NaN,81.0


### Export Cleaned Data to SQL
My data is now ready to be exported to my database. Thinking deeply about the dynamic nature of the column names, I realised the need to melt my data. One of the major reason is that If a "WHERE" clause is applied to a column in the database, and later the column name changes, this breaks the workflow and requires that the "WHERE" condition is manually rewritten, this would not be an effective way to handle this data, so I decided to melt the data. The staging table and main table has been created in SQL.

In [26]:
# Importing necessary libraries
import urllib
from sqlalchemy import create_engine, text

# Database Connection Parameters
params = (
    "DRIVER={ODBC Driver 17 for SQL Server};"
    "SERVER=*******\\SQLEXPRESS;"
    "DATABASE=DWETL;"
    "Trusted_Connection=yes;"
)

# Encoding the parameters for SQLAlchemy and creating a fast_executemany engine
quoted_params = urllib.parse.quote_plus(params)
connection_url = f"mssql+pyodbc:///?odbc_connect={quoted_params}"
engine = create_engine(connection_url, fast_executemany=True)

def transform_and_load_full(df):
    """
    1. Transposes (Melts) the wide Wikipedia DataFrame into Long format.
    2. Cleans metadata (Source/Year).
    3. Truncates the SQL table.
    4. Loads data using the SQLAlchemy engine.
    """
    print("Starting transformation...")

    # Melt/tranposing my data but "COUNTRY" stays as is
    value_cols = [c for c in df.columns if c != 'Country']
    
    df_long = df.melt(
        id_vars=['Country'], 
        value_vars=value_cols,
        var_name='Source_Year', 
        value_name='GDP_Value'
    )

    # Extracting source and year
    df_long['Source'] = df_long['Source_Year'].str.split('_').str[1]
    df_long['Data_Year'] = df_long['Source_Year'].str.split('_').str[2].astype(int)
    final_df = df_long[['Country', 'Source', 'Data_Year', 'GDP_Value']]
    
    # Remove any rows where Wikipedia had missing data (NaN)
    final_df = final_df.dropna(subset=['GDP_Value'])

    print(f"Transposed from {len(df)} rows to {len(final_df)} rows.")

    # Exporting to my database
    try:
        with engine.connect() as conn:
            print("Truncating staging table...")
            conn.execute(text("TRUNCATE TABLE dbo.StagingCountryGDP"))
            conn.commit()

        print("Uploading data to SQL Server...")
        final_df.to_sql(
            name='StagingCountryGDP', 
            con=engine, 
            if_exists='append', 
            index=False, 
            schema='dbo'
        )
        print("Success: Full script executed and data loaded in Long Format")
        
    except Exception as e:
        print(f"Error during SQL Load: {e}")

# Execute the process
if __name__ == "__main__":
    transform_and_load_full(CountriesGDP)

Starting transformation...
Transposed from 222 rows to 623 rows.
Truncating staging table...
Uploading data to SQL Server...
Success: Full script executed and data loaded in Long Format!


## Scheduling My Workflow Using Apache Airflow
The initial load has been completed. In the next step, an incremental refresh will be scheduled to capture every new information in the data. The SCD 2 historization is set to enable a proper tracking of the data update.
Airflow runs inside a Docker container, which gives a stable environment on Windows. The DAG (gdp_scd2_incremental_staging.py) automatically checks for new or changed rows and updates the staging table, so the data stays up-to-date without manual work.